# Realizzazione di una struttura rinforzata per il tetto

### RIPRODUZIONE DI UN MODELLO:

<img src="files/roof_model.jpg">

#### LINK: https://conceptdraw.com/a333c4/p1/preview/640/pict--landscape-design-garden-hotel-site-plan.png--diagram-flowchart-example.png

#### RIPRODUZIONE:

<img src="files/output_model.jpg"><img src="files/output_model2.jpg">

### UTILIZZO DELLA MEDESIMA FUNZIONE  PER LA RIPRODUZIONE DI TETTI E PINNACOLI DI NOTRE DAME DI PARIGI(approssimazione):

<img src="files/ND_real.jpg"><img src="files/ND_model.jpg">

#### LINK1: https://i.ytimg.com/vi/hFQt2Ajr7eQ/maxresdefault.jpg

#### LINK2: http://img10.deviantart.net/81a0/i/2011/038/2/9/notre_dame_top_view__by_hatredai-d390bdt.jpg

#### RIPRODUZIONE:

<img src="files/output_model3.jpg"><img src="files/output_model4.jpg"><img src="files/output_model5.jpg">

Importo le librerie necessarie

In [1]:
from pyplasm import *
from math import *

Evaluating fenvs.py..
...fenvs.py imported in 0.00545579815838 seconds


Definisco una funzione che data una lista di punti ne elimina gli errori di approssimazione

In [2]:
def noError (number):
    if number%0.0001 == 0.0:
        return number
    tmp = number*10000
    if tmp%1 >= .9:
        tmp = ceil(tmp)
    else:
        tmp = floor(tmp)
    return tmp/10000

Definisco una funzione che, dati un punto ed una lista di punti, verifica la presenza del punto nella lista.
Se il punto è presente mi restituisce l'indice del punto, altrimenti restituisce "-1"

In [3]:
def pointExists (point, pointsList):
    i = 0
    for p in pointsList:
        if abs(p[0] - point[0]) < 0.0001 and abs(p[1] - point[1]) < 0.0001 and abs(p[2] - point[2]) < 0.0001:
            return i
        i += 1
    return -1

Definisco una funzione che, data una lista di punti ed una lista di celle (caratteristiche di un tetto SOLIDO e PIENO), mi restituisce la relativa struttura rinforzata.
l'algoritmo consiste nel:
- realizzare un solido pieno a partire dai punti e dalle celle passati come parametri;
- estrapolare da questo solido tutte le celle piane grazie alla funzione SKEL_2;
- estrapolare tutti i punti (ridondanti) e tutte le celle utilizzando la funzione UKPOL;
- per ciascuna cella ne viene creata una nuova comprendente gli stessi punti più le proiezioni sul piano XY di tutti i punti "sopraelevati". Durante questa procedura vengono eliminate le ridondanze di punti e le possibili approssimazioni introdotte dal sistema (vedi funzione "noError");
- una volta ottenute due nuove liste (ottimizzate) di punti e celle, queste vengono utilizzate per creare un nuovo solido (che stavolta conterrà informazioni in più sui punti da rinforzare)
- si estrapola infine la struttura da questo nuovo solido utilizzando SKEL_1 e sucessivamente viene irrobustita con OFFSET

Si noti che la mia funzione prende come parametro anche la lista delle dimensioni utilizzati da OFFSET per permettere di realizzare strutture rinforzate in diverse scale (es. le travi del tetto-modello non sono come quelle di Notre Dame)

Ho commentato dei "print" di prova utili per analizzare le nuove liste di punti e celle realizzate


In [4]:
def ggpl_concrete_roof_builder (points, cells, offset):
    fullRoof = MKPOL([points, cells, 1])
    sk2 = SKEL_2(fullRoof)
    uk = UKPOL(sk2)
    uk_0 = uk[0]

    newPoints = []
    newCells = []
    for cell in uk[1]:
        tmp = []
        for index in cell:
            point = uk[0][index-1]
            point[0] = noError(point[0])
            point[1] = noError(point[1])
            point[2] = noError(point[2])
            i = pointExists(point, newPoints)
            #print i 
            if i > -1:
                tmp.append(i+1)
            else:
                newPoints.append(point)
                tmp.append(len(newPoints))
            if abs(point[2]) > 0.0001:
                newP = [point[0],point[1],0]
                j = pointExists(newP, newPoints)
                if j > -1:
                    tmp.append(j+1)
                else:
                    newPoints.append(newP)
                    tmp.append(len(newPoints))
        newCells.append(tmp)

    #print newPoints
    #print newCells
    new = MKPOL([newPoints, newCells, 1])
    sk = SKEL_1(new)
    roof = OFFSET(offset)(sk)
    return roof


Testo la funzione (tetto base, tetto a "L", tetto a "L" modificato)

In [5]:
points = [[0,0,0],[2,0,0],[2,4,0],[0,4,0],[1,1,1],[1,3,1]]
cells = [[1,2,3,4,5,6]]
#VIEW(ggpl_concrete_roof_builder(points, cells,[.1,.1,.2]))

points = [[0,0,0],[2,0,0],[1,1,2],[0,6,0],[2,4,0],[6,4,0],[6,6,0],[1,5,2],[5,5,2]]
cells =[[1,2,3,4,5,8],[4,5,6,7,8,9]]
#VIEW(ggpl_concrete_roof_builder(points, cells,[.1,.1,.2]))

points = [[0,0,0],[2,0,0],[1,1,2],[0,6,0],[2,4,0],[6,4,0],[6,6,0],[1,5,2],[5,5,4],[1,5,4]]
cells =[[1,2,3,4,5,8],[4,5,6,7,10,9]]
#VIEW(ggpl_concrete_roof_builder(points, cells,[.1,.1,.2]))

Utilizzo la funzione per riprodurre il modello del tetto (costruisco due strutture indipendenti e ne traslo una verso l'alto)

In [6]:
h0 = 3
h1 = 3
h2 = 2*h1
points = [[4,0,0],[28,0,0],[28,16,0],[4,16,0],[12,8,h2],[20,8,h2],
          [0,4,0],[4,4,0],[0,12,0],[4,12,0],[4,8,h1],[8,8,h1],
          [11,16,0],[11,22,0],[21,22,0],[21,16,0],[16,12,h1],[16,18,h1]]
cells =[[1,2,3,4,5,6],[7,8,9,10,11,12],[13,14,15,16,17,18]]
p = [[28,4,0],[28,12,0],[32,12,0],[32,4,0],[28,8,h0]]
c = [[1,2,3,4,5]]
        
VIEW(STRUCT([ggpl_concrete_roof_builder(p,c,[.1,.1,.2]), T(3)(6), ggpl_concrete_roof_builder(points,cells,[.1,.1,.2])]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000714D1B0> >

### Notre Dame roof

Utilizzo la funzione per riprodurre un tetto ottagonale

<img src="files/octagon.jpg" width="300">

In [7]:
l = 5
t = (l/2)*sqrt(2)
h = 10*l
pt = [[l,0,0], [t, -t,0],[0,-l,0],[-t, -t,0],[-l,0,0], [-t, t,0], [0,l,0], [t, t,0], [0,0,h]]
for p in pt:
    p[0] += 115
    p[1] += 35
    
cl = [[1,2,3,4,5,6,7,8,9]]
pinnacleRoof = ggpl_concrete_roof_builder(pt,cl,[1,1,2])

Con gli stessi punti di base riproduco una torre ottagonale

In [8]:
pt2 = [[l,0], [t, -t],[0,-l],[-t, -t],[-l,0], [-t, t], [0,l], [t, t]]
for p in pt2:
    p[0] += 115
    p[1] += 35

tower_2d = MKPOL([pt2,[[1,2,3,4,5,6,7,8]],1])
hTower = 10
tower_3d =  OFFSET([1,1,2])(SKEL_1(PROD([tower_2d,QUOTE([hTower])])))


Riproduco la struttura delle mura principali ed una torre quadrata

In [9]:
hWalls = 50
structureNDcentral = OFFSET([1,1,2])(SKEL_1(MKPOL([[[0,30,0],[0,30,hWalls],[0,40,0],[0,40,hWalls],
      [110,0,0],[110,0,hWalls],[120,0,0],[120,0,50],
      [110,70,0],[110,70,hWalls],[120,70,0],[120,70,hWalls],
      [180,30,0],[180,30,hWalls],[180,40,0],[180,40,hWalls],
      [110,30,0],[110,30,hWalls],[120,30,0],[120,30,hWalls],
      [110,40,0],[110,40,hWalls],[120,40,0],[120,40,hWalls],
      ],
      [[17,18,19,20,21,22,23,24],
      [1,2,3,4,17,18,21,22],
      [5,6,7,8,17,18,19,20],
      [13,14,15,16,19,20,23,24],
      [21,22,23,24,9,10,11,12]],1])))

normalTower = OFFSET([1,1,2])(SKEL_1(CUBOID([20,20,90])))


Assemblo la struttura finale aggiungendo nell'ordine:
- mura
- torre ottagonale
- pinnacolo (tetto ottagonale)
- torri
- tetti ottagonali bassi
- tetti restanti

In [10]:
structureFinal = STRUCT([structureNDcentral,T(3)(hWalls),tower_3d,T(3)(hTower),pinnacleRoof])

structureFinal = STRUCT([structureFinal,T(2)(10),normalTower,T(2)(30),normalTower])

l = 8
t = (l/2)*sqrt(2)
h = l
pt = [[l,0,0], [t, -t,0],[0,-l,0],[-t, -t,0],[-l,0,0], [-t, t,0], [0,l,0], [t, t,0], [0,0,h]]
for p in pt:
    p[0] += 10
    
cl = [[1,2,3,4,5,6,7,8,9]]
pinnacleShortRoof = ggpl_concrete_roof_builder(pt,cl,[1,1,2])

structureFinal = STRUCT([structureFinal,T([2,3])([20,90]),pinnacleShortRoof,T([2])([30]),pinnacleShortRoof])

roofWest = ggpl_concrete_roof_builder([[20,30,0],[20,40,0],[110,40,0],[110,30,0],[20,35,10],[110,35,10]],[[1,2,3,4,5,6]],[1,1,2])
roofEast = ggpl_concrete_roof_builder([[120,30,0],[120,40,0],[180,40,0],[180,30,0],[170,35,10],[120,35,10]],[[1,2,3,4,5,6]],[1,1,2])
roofSouth = ggpl_concrete_roof_builder([[110,0,0],[120,0,0],[110,30,0],[120,30,0],[115,0,10],[115,30,10]],[[1,2,3,4,5,6]],[1,1,2])
roofNorth = ggpl_concrete_roof_builder([[110,70,0],[120,70,0],[110,40,0],[120,40,0],[115,70,10],[115,40,10]],[[1,2,3,4,5,6]],[1,1,2])

structureFinal = STRUCT([structureFinal,T(3)(hWalls),roofWest,roofEast,roofSouth,roofNorth])

VIEW(structureFinal)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x0000000007167600> >